<a href="https://colab.research.google.com/github/ljsantos/MiyabaraThiago_DataEngineeringScience/blob/master/NotebooksDataScience/Studies/MyBeginningKaggle/GRU_Airport_Stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#!pip install selenium
!ls -al /usr/lib/chromium-browser/chromedriver

total 198116
drwxr-xr-x 6 root root      4096 Nov  9 14:18 .
drwxr-xr-x 1 root root      4096 Nov  9 14:18 ..
-rw-r--r-- 1 root root     23125 Nov  1 09:09 browser_tests_apprtc.runtime_deps
-rwxr-xr-x 1 root root    272568 Nov  1 09:09 bytecode_builtins_list_generator
-rwxr-xr-x 1 root root    366784 Nov  1 09:09 cddl
-rw-r--r-- 1 root root   1064459 Nov  1 09:09 chrome_100_percent.pak
-rw-r--r-- 1 root root   1377696 Nov  1 09:09 chrome_200_percent.pak
lrwxrwxrwx 1 root root        22 Nov  1 09:09 chromedriver -> ../../bin/chromedriver
-r-sr-xr-x 1 root root    243896 Nov  1 09:09 chrome-sandbox
-rwxr-xr-x 1 root root 155217144 Nov  1 09:09 chromium-browser
-rw-r--r-- 1 root root   8781786 Nov  1 09:09 headless_lib.pak
-rw-r--r-- 1 root root  10426784 Oct 21 19:09 icudtl.dat
-rw-r--r-- 1 root root   3055528 Nov  1 09:09 libffmpeg.so
lrwxrwxrwx 1 root root         1 Nov  1 09:09 libs -> .
-rw-r--r-- 1 root root    317496 Nov  1 09:09 libVkICD_mock_icd.so
drwxr-xr-x 2 root root      409

In [0]:
from urllib.request import Request,urlopen
import pandas as pd
from selenium import webdriver
import time
import os

if os.name == 'nt':
    browser = webdriver.Firefox(executable_path='E:/ZProjetos/geckodriver/geckodriver.exe')
else:
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('disable-dev-shm-usage')
    browser = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=options)

browser.get('https://www.gru.com.br/pt/passageiro/descubra-gru/lojas/todos-os-estabelecimentos')
time.sleep(20)

#table = browser.find_element_by_class_name('estabelecimento')
table = browser.find_element_by_id("tabela-0")

table_html = table.get_attribute('outerHTML')

df = pd.read_html(str(table_html))

#print(df)
df = df[0]

In [0]:
lista_de_topicos = table.find_elements_by_xpath("//table/tbody/tr/th/header/h2")

In [117]:
lista_de_topicos[0].text

'Alimentação'

In [0]:
lista_de_tabelas_locais = table.find_elements_by_xpath("//table/tbody/tr/td/div")

In [119]:
i=0
estabelecimento=lista_de_tabelas_locais[i].find_elements_by_tag_name('h3')[0].text
estabelecimento

'365 Deli'

In [120]:
tabela=pd.read_html(lista_de_tabelas_locais[i].find_elements_by_tag_name('table')[0].get_attribute('outerHTML'))
tabela

[            0                     1          2               3              4
 0  Terminal 2  Conector de Embarque        24h   (11)2445-4637  Área Restrita
 1  Terminal 3              Embarque        24h   (11)2445-4637  Área Restrita
 2  Terminal 3              Mezanino        24h  (11) 2445-4637  Área Restrita
 3  Terminal 2            Píer Oeste        24h  (11) 2445-4637  Área Restrita
 4  Terminal 3       Embarque Remoto  06h - 23h             NaN  Área Restrita]

In [121]:
tabela_df = tabela[0]
tabela_df

,0,1,2,3,4
0,Terminal 2,Conector de Embarque,24h,(11)2445-4637,Área Restrita
1,Terminal 3,Embarque,24h,(11)2445-4637,Área Restrita
2,Terminal 3,Mezanino,24h,(11) 2445-4637,Área Restrita
3,Terminal 2,Píer Oeste,24h,(11) 2445-4637,Área Restrita
4,Terminal 3,Embarque Remoto,06h - 23h,NaN,Área Restrita


In [0]:
lista_nivel1 = table.find_elements_by_xpath("//table/tbody/tr")

In [134]:
len(lista_nivel1)

419

In [152]:
lista_nivel1[0].find_elements_by_xpath("./th/header/h2")

[<selenium.webdriver.remote.webelement.WebElement (session="674455386a809913fe2c50db9d7162ed", element="3aae26fb-3499-406e-a1ac-de55a6c9a652")>]

In [0]:
topico = ""
tabela_local_html = ""
loja = ""
novas_colunas = {0: "terminal", 1: "local", 2: "horario", 3: "telefone", 4: "tipo de acesso"}
colunas_ordenadas = ['topico', 'loja', 'terminal', 'local', 'horario', 'telefone', 'tipo de acesso']
del tabela_local
del tabela_final

for item in lista_nivel1:
    topico_html = item.find_elements_by_xpath("./th/header/h2")
    if topico_html:
        topico = topico_html[0].text

    tabela_local_html = item.find_elements_by_xpath("./td/div")
    if tabela_local_html:
        loja = tabela_local_html[0].find_elements_by_tag_name('h3')[0].text
        tabela_local = pd.read_html(tabela_local_html[0].find_elements_by_tag_name('table')[0].get_attribute('outerHTML'))[0]
        tabela_local = tabela_local.rename(novas_colunas, axis='columns')
        tabela_local = tabela_local.assign(topico=topico, loja=loja)
        tabela_local = tabela_local[colunas_ordenadas]

        try:
            tabela_final = tabela_final.append(tabela_local, sort=False, ignore_index=True)
        except:
            tabela_final = tabela_local.copy()
 

In [202]:
display(tabela_final)

,topico,loja,terminal,local,horario,telefone,tipo de acesso
0,Alimentação,365 Deli,Terminal 2,Conector de Embarque,24h,(11)2445-4637,Área Restrita
1,Alimentação,365 Deli,Terminal 3,Embarque,24h,(11)2445-4637,Área Restrita
2,Alimentação,365 Deli,Terminal 3,Mezanino,24h,(11) 2445-4637,Área Restrita
3,Alimentação,365 Deli,Terminal 2,Píer Oeste,24h,(11) 2445-4637,Área Restrita
4,Alimentação,365 Deli,Terminal 3,Embarque Remoto,06h - 23h,NaN,Área Restrita
...,...,...,...,...,...,...,...
246,Lojas,Vivara,Terminal 2,Embarque Leste,06h - 23h,NaN,Restrita
247,Lojas,Vivara,Terminal 3,Píer de Embarque,07h - 23h,NaN,Restrita
248,Lojas,WH Smith,Terminal 2,Leste,06h - 23h,NaN,Restrita
249,Lojas,WH Smith,Terminal 1,Embarque,06h - 23h,NaN,Restrita


In [204]:
tabela_final.shape

(251, 7)